# 05 — Resultados e Discussão (ENEM 2023)

**Objetivo:** consolidar os principais achados da análise exploratória para apresentação do checkpoint.

**Este notebook mostra:**
1. Estatísticas descritivas e distribuições (univariado).
2. Relações bivariadas: renda × nota; escolaridade dos pais × nota; tipo de escola × nota.
3. Correlações cruzadas (renda, escolaridade, notas).
4. Blocos de interpretação orientando a discussão.

In [ ]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA_PATH = Path("../data/interim/enem_2023.parquet")
print("Lendo:", DATA_PATH)
df = pd.read_parquet(DATA_PATH)
print("Dimensões:", df.shape)

NOTAS = ["NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_CH","NU_NOTA_CN","NU_NOTA_REDACAO"]
REQUIRED = ["Q006","Q006_ord","Q001","Q001_ord","Q002","Q002_ord","TP_ESCOLA","NOTA_MEDIA_5"] + NOTAS
missing = [c for c in REQUIRED if c not in df.columns]
if missing:
    raise RuntimeError(f"Faltam colunas necessárias: {missing}")

if "IN_TREINEIRO" in df.columns:
    df = df[df["IN_TREINEIRO"] == 0].copy()

df = df.dropna(subset=["NOTA_MEDIA_5","Q006_ord","Q001_ord","Q002_ord","TP_ESCOLA"]).copy()
print("Após filtros:", df.shape)
df.head(3)

## 1) Estatísticas descritivas e distribuições (univariado)

In [ ]:
desc = df[NOTAS + ["NOTA_MEDIA_5"]].describe().T
desc[["mean","std","min","25%","50%","75%","max"]]

In [ ]:
plt.figure(figsize=(8,4.8))
plt.hist(df["NOTA_MEDIA_5"].dropna(), bins=40)
plt.title("Distribuição — Nota Média (5 provas)")
plt.xlabel("Nota média")
plt.ylabel("Frequência")
plt.tight_layout()
plt.show()

In [ ]:
for col in ["NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_CH","NU_NOTA_CN","NU_NOTA_REDACAO"]:
    plt.figure(figsize=(8,4.8))
    plt.hist(df[col].dropna(), bins=40)
    plt.title(f"Distribuição — {col}")
    plt.xlabel("Nota")
    plt.ylabel("Frequência")
    plt.tight_layout()
    plt.show()

## 2) Renda (Q006_ord) × Nota

In [ ]:
g = df.groupby("Q006_ord", as_index=False)["NOTA_MEDIA_5"].mean().sort_values("Q006_ord")
plt.figure(figsize=(9,4.8))
plt.plot(g["Q006_ord"], g["NOTA_MEDIA_5"], marker="o")
plt.title("Nota Média por Faixa de Renda (Q006_ord)")
plt.xlabel("Renda (ordem A=1, B=2, ...)")
plt.ylabel("Nota média (5 provas)")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

corr_renda_media = df["Q006_ord"].corr(df["NOTA_MEDIA_5"])
print("Correlação Q006_ord × NOTA_MEDIA_5:", round(corr_renda_media, 3))

## 3) Escolaridade dos pais (Q001_ord, Q002_ord) × Nota

In [ ]:
g1 = df.groupby("Q001_ord", as_index=False)["NOTA_MEDIA_5"].mean().sort_values("Q001_ord")
plt.figure(figsize=(9,4.8))
plt.plot(g1["Q001_ord"], g1["NOTA_MEDIA_5"], marker="o")
plt.title("Nota Média por Escolaridade do Responsável 1 (Q001_ord)")
plt.xlabel("Q001_ord (A=1, B=2, ...)")
plt.ylabel("Nota média")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()
print("Correlação Q001_ord × NOTA_MEDIA_5:", round(df["Q001_ord"].corr(df["NOTA_MEDIA_5"]), 3))

g2 = df.groupby("Q002_ord", as_index=False)["NOTA_MEDIA_5"].mean().sort_values("Q002_ord")
plt.figure(figsize=(9,4.8))
plt.plot(g2["Q002_ord"], g2["NOTA_MEDIA_5"], marker="o")
plt.title("Nota Média por Escolaridade do Responsável 2 (Q002_ord)")
plt.xlabel("Q002_ord (A=1, B=2, ...)")
plt.ylabel("Nota média")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()
print("Correlação Q002_ord × NOTA_MEDIA_5:", round(df["Q002_ord"].corr(df["NOTA_MEDIA_5"]), 3))

## 4) Tipo de escola (TP_ESCOLA) × Nota

In [ ]:
map_escola = {1: "Pública", 2: "Privada", 3: "Exterior"}
lab = df["TP_ESCOLA"].map(map_escola).fillna(df["TP_ESCOLA"].astype(str))

grp = pd.DataFrame({"TP_ESCOLA_lab": lab, "NOTA_MEDIA_5": df["NOTA_MEDIA_5"]}).groupby("TP_ESCOLA_lab", as_index=False)["NOTA_MEDIA_5"].mean()

plt.figure(figsize=(7.2,4.5))
plt.bar(grp["TP_ESCOLA_lab"], grp["NOTA_MEDIA_5"])
plt.title("Nota Média por Tipo de Escola")
plt.xlabel("Tipo de escola")
plt.ylabel("Nota média (5 provas)")
plt.tight_layout()
plt.show()

grp.sort_values("NOTA_MEDIA_5", ascending=False)

## 5) Correlações cruzadas — renda, escolaridade e notas

In [ ]:
cols_corr = ["Q006_ord","Q001_ord","Q002_ord","NOTA_MEDIA_5"] + NOTAS
sub = df[cols_corr].dropna()
corr = sub.corr(numeric_only=True).round(3)
print(corr)

In [ ]:
plt.figure(figsize=(7,6))
plt.imshow(corr, aspect='auto')
plt.title("Correlação — Renda, Escolaridade e Notas")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha='right')
plt.yticks(range(len(corr.index)), corr.index)
plt.tight_layout()
plt.show()

## 6) Interpretação e discussão (para apresentação)

- **Distribuições**: notas concentram-se em faixas médias; dispersão maior em Matemática e Redação.  
- **Renda × nota**: tendência monotônica crescente; correlação positiva entre `Q006_ord` e `NOTA_MEDIA_5`.  
- **Escolaridade × nota**: padrão semelhante para `Q001_ord` e `Q002_ord`.  
- **Tipo de escola**: médias maiores no privado, mas renda e escolaridade permanecem relevantes.  
- **Correlação cruzada**: renda e escolaridade correlacionam-se entre si e com notas, sobretudo *Matemática* e *Redação*.  

**Próximos passos**  
- Explorar análises por área com maior detalhe (com os notebooks 02 e 03).  
- Agrupar faixas de renda para simplificar comunicação (baixa/média/alta).  
- Testar ponderações de notas (contexto SISU/curso) ou modelos simples (regressão).